# Develop Generative AI Solutions in Azure

## Use a Prompt Flow to Manage Conversation in a Chat App

**Purpose:** Use Azure AI Foundry Prompt Flow to design, test, and deploy a custom chat experience that incorporates user input and chat history with a GPT model.

**Estimated Time:** ~30 minutes
**Status:** Some features are preview / in active development

---

## 1. Create a Foundry Hub and Project

> Prompt Flow requires a **Foundry hub–based project**, not a project based only on a Foundry resource.

### Create the Hub-Based Project

1. Open **[https://ai.azure.com](https://ai.azure.com)** and sign in.
2. Close tips and help panes.
3. Navigate to:

   * `https://ai.azure.com/managementCenter/allResources`
4. Select **Create new** → **AI hub resource**.

### Project Configuration

In the **Create a project** wizard:

* Project name: Valid name
* Hub: **Create new hub** → Rename to a unique alphanumeric name
* Expand **Advanced options**

Specify:

* **Subscription:** Your Azure subscription
* **Resource group:** Create or select
* **Region:** East US 2 or Sweden Central

> If quotas are exceeded later, you may need a different region.

**Important:** If *Create* is disabled, ensure the hub name is unique.

Wait for the project and hub to be created.

---

## 2. Configure Resource Authorization

Prompt flows store assets in **Blob Storage**, so the Foundry resource must have permission to read blobs.

### Enable Managed Identity

1. Open **[https://portal.azure.com](https://portal.azure.com)** in a new tab.
2. Navigate to the **resource group** for your AI hub.
3. Select the **Foundry resource**.
4. Open **Identity** under *Resource Management*.
5. Set **System-assigned identity** to **On**.
6. Save and wait for confirmation.

---

### Assign Storage Permissions

1. Return to the resource group.
2. Select the **Storage account** for the hub.
3. Open **Access Control (IAM)**.
4. Add a role assignment:

   * **Role:** Storage Blob Data Reader
   * **Assignee:** Foundry resource managed identity

This allows Prompt Flow to read flow assets from storage.

Return to the Foundry portal.

---

## 3. Deploy a Generative AI Model

### Deploy gpt-4o

1. In the project navigation pane, select **Models + endpoints**.
2. Open the **Model deployments** tab.
3. Select **+ Deploy model** → **Deploy base model**.
4. Search for **gpt-4o** and confirm.

### Deployment Settings

Select **Customize** and configure:

* **Deployment name:** Valid name
* **Deployment type:** Global Standard
* **Automatic version update:** Enabled
* **Model version:** Most recent
* **Connected AI resource:** Azure OpenAI connection
* **Tokens per minute:** 50K (or max available)
* **Content filter:** DefaultV2

> Lower TPM helps prevent quota exhaustion.

Wait for deployment to complete.

---

## 4. Create a Prompt Flow

### Create the Flow

1. In the Foundry navigation bar, select **Build and customize** → **Prompt flow**.
2. Create a new flow using the **Chat flow** template.
3. Set folder name to **Travel-Chat**.

A basic chat flow is generated.

> If a permissions error occurs, wait a few minutes and retry with a different flow name.

---

### Start Compute

* Select **Start compute session** immediately
* Compute startup can take several minutes

---

### Review Flow Components

* **Inputs:**

  * `question`
  * `chat_history`
* **Outputs:**

  * Model-generated answer
* **Chat LLM Tool:**

  * Handles prompt submission to the model

---

### Configure Chat LLM Tool

Set the following properties:

* **Connection:** Azure OpenAI resource connection
* **Api:** chat
* **deployment_name:** gpt-4o deployment
* **response_format:** `{ "type": "text" }`

---

### Update the Prompt

Replace the Prompt field with the following (remove YAML escape characters):

```
# system:
**Objective**: Assist users with travel-related inquiries, offering tips, advice, and recommendations as a knowledgeable travel agent.

**Capabilities**:
- Provide up-to-date travel information
- Offer personalized travel suggestions
- Share packing, safety, and disruption tips
- Help plan itineraries and routes
- Answer common travel questions

**Instructions**:
1. Be friendly and professional
2. Provide accurate and relevant info
3. Tailor responses to user needs
4. Consider safety and comfort
5. Encourage follow-up questions

{% for item in chat_history %}
# user:
{{item.inputs.question}}
# assistant:
{{item.outputs.answer}}
{% endfor %}

# user:
{{question}}
```

### Verify Input Bindings

Ensure:

* `question` → `${inputs.question}`
* `chat_history` → `${inputs.chat_history}`

Save the flow.

---

## 5. Test the Flow

1. Ensure the compute session is running.
2. Select **Chat** from the toolbar.
3. Wait for initialization.
4. Enter:

   * *I have one day in London, what should I do?*
5. Review the response.

The chat should incorporate both system instructions and history.

---

## 6. Deploy the Flow

### Deployment Settings

Select **Deploy** and configure:

**Basic:**

* Endpoint: New
* Endpoint name: Unique
* Deployment name: Unique
* VM: Standard_DS3_v2
* Instance count: 1
* Inferencing data collection: Disabled

**Advanced:**

* Use defaults

> Deployment may take significant time due to capacity constraints.

---

### Verify Deployment

1. Navigate to **Models + endpoints**.
2. Use back navigation if needed to view all endpoints.
3. Wait until deployment status is successful.
4. Open the endpoint **Test** page.

If endpoint shows *unhealthy*, wait and refresh.

### Test Prompts

* *What is there to do in San Francisco?*
* *Tell me something about the history of the city.*

---

### Consume the Endpoint

* Open the **Consume** page
* Review endpoint URL, auth details, and sample code
* Use this information to integrate the flow into an application

---

## 7. Clean Up

To avoid Azure charges:

1. Open **[https://portal.azure.com](https://portal.azure.com)**
2. Go to **Resource groups**
3. Select the group used for this lab
4. Choose **Delete resource group**
5. Confirm deletion

---

## Quick Exam Takeaways

* Prompt Flow requires a **hub-based project**
* Managed identity + blob access is required
* Chat history must be explicitly included in prompts
* Prompt Flow supports orchestration beyond simple chat
* Deployed flows expose endpoints like models
* Always clean up hub, storage, and compute resources
